In [1]:
# Importing libraries
import pandas as pd
import nltk
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Importing the module from nltk for stemming words 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [3]:
# Loading the dataset into a dataframe
df = pd.read_csv(r"C:\Users\dines\OneDrive\Desktop\projects\recommenderSystem\Coursera.csv")
df.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [4]:
# Getting the shape of the dataset
df.shape

(3522, 7)

In [5]:
# Getting the information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


In [6]:
# Getting the null values of the dataset
df.isnull().sum()

Course Name           0
University            0
Difficulty Level      0
Course Rating         0
Course URL            0
Course Description    0
Skills                0
dtype: int64

Performing EDA

In [7]:
df['Difficulty Level'].value_counts()

Beginner          1444
Advanced          1005
Intermediate       837
Conversant         186
Not Calibrated      50
Name: Difficulty Level, dtype: int64

In [8]:
df['Course Rating'].value_counts()

4.7               740
4.6               623
4.8               598
4.5               389
4.4               242
4.9               180
4.3               165
4.2               121
5                  90
4.1                85
Not Calibrated     82
4                  51
3.8                24
3.9                20
3.6                18
3.7                18
3.5                17
3.4                13
3                  12
3.2                 9
3.3                 6
2.9                 6
2.6                 2
2.8                 2
2.4                 2
1                   2
2                   1
3.1                 1
2.5                 1
1.9                 1
2.3                 1
Name: Course Rating, dtype: int64

In [9]:
df['University'].value_counts()

Coursera Project Network                      562
University of Illinois at Urbana-Champaign    138
Johns Hopkins University                      110
University of Michigan                        101
University of Colorado Boulder                101
                                             ... 
GitLab                                          1
Yeshiva University                              1
University of Glasgow                           1
Laureate Education                              1
The World Bank Group                            1
Name: University, Length: 184, dtype: int64

In [10]:
df['Course Name']

0       Write A Feature Length Screenplay For Film Or ...
1       Business Strategy: Business Model Canvas Analy...
2                           Silicon Thin Film Solar Cells
3                                    Finance for Managers
4            Retrieve Data using Single-Table SQL Queries
                              ...                        
3517    Capstone: Retrieving, Processing, and Visualiz...
3518                     Patrick Henry: Forgotten Founder
3519    Business intelligence and data analytics: Gene...
3520                                  Rigid Body Dynamics
3521    Architecting with Google Kubernetes Engine: Pr...
Name: Course Name, Length: 3522, dtype: object

In [11]:
# Dropping the unnecessary columns as they are not required for the recommender system
df = df[['Course Name','Difficulty Level','Course Description','Skills']]

In [12]:
# Renaming the columns
df.columns = ['name','difficulty level','description','skills']
df.head()

,name,difficulty level,description,skills
0,Write A Feature Length Screenplay For Film Or ...,Beginner,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Beginner,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,Advanced,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,Intermediate,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Beginner,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [29]:
# Tokenizing the columns 
df['name'] = df['name'].str.replace(' ',',',regex=True)
df['name'] = df['name'].str.replace(',,',',',regex=True)
df['name'] = df['name'].str.replace(':','',regex=True)
df['description'] = df['description'].str.replace(' ',',',regex=True)
df['description'] = df['description'].str.replace(',,',',',regex=True)
df['description'] = df['description'].str.replace('_','',regex=True)
df['description'] = df['description'].str.replace(':','',regex=True)
df['description'] = df['description'].str.replace('(','',regex=True)
df['description'] = df['description'].str.replace(')','',regex=True)

df['skills'] = df['skills'].str.replace('(','',regex=True)
df['skills'] = df['skills'].str.replace(')','',regex=True)
df.head()

,name,difficulty level,description,skills,tags
0,"Write,A,Feature,Length,Screenplay,For,Film,Or,...",Beginner,"Write,a,Full,Length,Feature,Film,Script,In,thi...",Drama Comedy peering screenwriting film D...,"Write,A,Feature,Length,Screenplay,For,Film,Or,..."
1,"Business,Strategy,Business,Model,Canvas,Analys...",Beginner,"By,the,end,of,this,guided,project,you,will,be,...",Finance business plan persona user experienc...,"Business,Strategy,Business,Model,Canvas,Analys..."
2,"Silicon,Thin,Film,Solar,Cells",Advanced,"This,course,consists,of,a,general,presentation...",chemistry physics Solar Energy film lambda...,"Silicon,Thin,Film,Solar,Cells Advanced This,co..."
3,"Finance,for,Managers",Intermediate,"When,it,comes,to,numbers,there,is,always,more,...",accounts receivable dupont analysis analysis...,"Finance,for,Managers Intermediate When,it,come..."
4,"Retrieve,Data,using,Single-Table,SQL,Queries",Beginner,"In,this,course,you�ll,learn,how,to,effectively...",Data Analysis select sql database management...,"Retrieve,Data,using,Single-Table,SQL,Queries B..."


In [14]:
# concatenating the columns into a single column 
df['tags'] = df['name'] +' '+ df['difficulty level'] +' '+ df['description'] +' '+ df['skills']
df['tags'][0]

'Write,A,Feature,Length,Screenplay,For,Film,Or,Television Beginner Write,a,Full,Length,Feature,Film,Script,In,this,course,you,will,write,a,complete,feature-length,screenplay,for,film,or,television,be,it,a,serious,drama,or,romantic,comedy,or,anything,in,between.,You�ll,learn,to,break,down,the,creative,process,into,components,and,you�ll,discover,a,structured,process,that,allows,you,to,produce,a,polished,and,pitch-ready,script,by,the,end,of,the,course.,Completing,this,project,will,increase,your,confidence,in,your,ideas,and,abilities,and,you�ll,feel,prepared,to,pitch,your,first,script,and,get,started,on,your,next.,This,is,a,course,designed,to,tap,into,your,creativity,and,is,based,in,"Active,Learning".,Most,of,the,actual,learning,takes,place,within,your,own,activities,-,that,is,writing!,You,will,learn,by,doing.,Here,is,a,link,to,a,TRAILER,for,the,course.,To,view,the,trailer,please,copy,and,paste,the,link,into,your,browser.,https//vimeo.com/382067900/b78b800dc0,Learner,review,"Love,the,appro

In [33]:
# Dropping the unnecessary columns 
new_df = df[['name','tags']].copy()
new_df.head()

,name,tags
0,"Write,A,Feature,Length,Screenplay,For,Film,Or,...","Write,A,Feature,Length,Screenplay,For,Film,Or,..."
1,"Business,Strategy,Business,Model,Canvas,Analys...","Business,Strategy,Business,Model,Canvas,Analys..."
2,"Silicon,Thin,Film,Solar,Cells","Silicon,Thin,Film,Solar,Cells Advanced This,co..."
3,"Finance,for,Managers","Finance,for,Managers Intermediate When,it,come..."
4,"Retrieve,Data,using,Single-Table,SQL,Queries","Retrieve,Data,using,Single-Table,SQL,Queries B..."


In [35]:
# Removing commas from the tags column
new_df['tags'] = new_df['tags'].str.replace(',',' ')

In [36]:
# Removing commas from the name column
new_df['name'] = new_df['name'].str.replace(',',' ')

In [37]:
# Converting the tags column into lower case
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head()

,name,tags
0,Write A Feature Length Screenplay For Film Or ...,write a feature length screenplay for film or ...
1,Business Strategy Business Model Canvas Analys...,business strategy business model canvas analys...
2,Silicon Thin Film Solar Cells,silicon thin film solar cells advanced this co...
3,Finance for Managers,finance for managers intermediate when it come...
4,Retrieve Data using Single-Table SQL Queries,retrieve data using single-table sql queries b...


In [38]:
# getting the shape of the new dataframe
new_df.shape

(3522, 2)

In [39]:
# Importing the libraries for text vectorization
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
# Creating an instance of the CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [41]:
# Fitting the CountVectorizer to the tags column
vectors = cv.fit_transform(new_df['tags']).toarray()

In [42]:
#defining the stemming function
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [43]:
# reducing the words to their root words
new_df['tags'] = new_df['tags'].apply(stem)

In [44]:
# Finding the cosine similarity between the vectors
similarity = cosine_similarity(vectors)

In [45]:
# Recommending the courses based on the course name
def recommend(course):
    course_index = new_df[new_df['name'] == course].index[0]
    distances = similarity[course_index]
    course_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:7]
    
    for i in course_list:
        print(new_df.iloc[i[0]]['name'])